In [1]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import spacy
from tqdm import tqdm  # Import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Load your dataset (replace 'your_dataset.csv' with the actual file path)
df = pd.read_csv(r'E:\SNU Chennai\projects\NLP project\NEWS classification\news article\news-article-categories.csv')

# Combine 'title' and 'body' columns into a single 'text' column
df['text'] = df['title'] + " " + df['body']

# Drop rows where 'text' is missing
df = df.dropna(subset=['text'])

c:\Users\sreen\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Preprocessing function using spaCy
def preprocess_text(text):
    # Process text with spaCy
    doc = nlp(text.lower())
    
    # Remove stopwords and non-alphabetic tokens, then lemmatize
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    
    # Join back into a string
    return " ".join(tokens)

# Apply preprocessing to the 'text' column with tqdm
print("Preprocessing text...")
tqdm.pandas()  # Enable tqdm progress_apply
df['clean_text'] = df['text'].progress_apply(preprocess_text)  # Use progress_apply instead of apply

Preprocessing text...


100%|██████████| 6872/6872 [06:34<00:00, 17.44it/s]


In [3]:
# Encode labels as integers
label_to_id = {label: idx for idx, label in enumerate(set(df['category']))}
id_to_label = {idx: label for label, idx in label_to_id.items()}
df['label_encoded'] = df['category'].apply(lambda x: label_to_id[x])

In [4]:
# Define a custom dataset for PyTorch
class NewsDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = self.embeddings[idx]
        label = self.labels[idx]
        return torch.tensor(embedding, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Custom collate function to handle padding
def collate_fn(batch):
    embeddings, labels = zip(*batch)
    embeddings = torch.nn.utils.rnn.pad_sequence(
        [emb[:512] for emb in embeddings],  # Ensure no sequence exceeds max_length
        batch_first=True,
        padding_value=0.0
    )
    labels = torch.tensor(labels, dtype=torch.long)
    return embeddings, labels

In [5]:
# Define the BiLSTM model
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, dropout=0.5):
        super(BiLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiply by 2 for bidirectional

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        avg_pool = torch.mean(lstm_out, dim=1)  # Global average pooling
        out = self.dropout(avg_pool)
        out = self.fc(out)
        return out

# Hyperparameters
input_dim = 768  # BERT embedding size
hidden_dim = 64  # Changed to 64 units as per the description
output_dim = len(label_to_id)
num_layers = 1  # Single-layer LSTM
dropout = 0.5
learning_rate = 0.001
num_epochs = 100  # Increased to 100 epochs
batch_size = 16

In [ ]:
# Load pre-trained BERT tokenizer and model
print("Loading pre-trained BERT model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Function to compute token-level embeddings using BERT
def get_token_embeddings(text, tokenizer, model, max_length=512):
    # Tokenize the text
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',  # Pad to max_length
        max_length=max_length
    )
    
    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Return token embeddings (last_hidden_state)
    return outputs.last_hidden_state.squeeze(0).numpy()

# Convert documents to token embeddings
print("Converting data to BERT token embeddings...")
X_embeddings = [get_token_embeddings(text, tokenizer, bert_model) for text in tqdm(df['clean_text'])]
y_labels = df['label_encoded'].values

Loading pre-trained BERT model...
Converting data to BERT token embeddings...


100%|██████████| 6872/6872 [29:26<00:00,  3.89it/s]


: 

In [ ]:
# 5-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X_embeddings), 1):
    print(f"Training on Fold {fold}...")
    
    # Split data into training and testing sets for this fold
    X_train_embeddings = [X_embeddings[i] for i in train_idx]
    X_test_embeddings = [X_embeddings[i] for i in test_idx]
    y_train_encoded = y_labels[train_idx]
    y_test_encoded = y_labels[test_idx]

    # Create datasets and dataloaders
    train_dataset = NewsDataset(X_train_embeddings, y_train_encoded)
    test_dataset = NewsDataset(X_test_embeddings, y_test_encoded)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # Initialize the model, loss function, and optimizer
    model = BiLSTMClassifier(input_dim, hidden_dim, output_dim, num_layers, dropout)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for embeddings, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            optimizer.zero_grad()
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Fold {fold}, Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

    # Evaluate the model
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for embeddings, labels in tqdm(test_loader, desc="Testing"):
            outputs = model(embeddings)
            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"\nFold {fold}, Accuracy: {accuracy}")
    fold_accuracies.append(accuracy)

# Final Results
print("\nFinal Results:")
print(f"Average Accuracy across 5 folds: {np.mean(fold_accuracies):.4f}")

Training on Fold 1...


Epoch 1/100:   0%|          | 0/344 [00:00<?, ?it/s]

In [5]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import spacy
from tqdm import tqdm  # Import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, Dataset

# Check for GPU availability and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Load your dataset (replace 'your_dataset.csv' with the actual file path)
df = pd.read_csv(r'E:\SNU Chennai\projects\NLP project\NEWS classification\news article\news-article-categories.csv')

# Combine 'title' and 'body' columns into a single 'text' column
df['text'] = df['title'] + " " + df['body']

# Drop rows where 'text' is missing
df = df.dropna(subset=['text'])

# Preprocessing function using spaCy
def preprocess_text(text):
    # Process text with spaCy
    doc = nlp(text.lower())
    
    # Remove stopwords and non-alphabetic tokens, then lemmatize
    tokens = [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]
    
    # Join back into a string
    return " ".join(tokens)

# Apply preprocessing to the 'text' column with tqdm
print("Preprocessing text...")
tqdm.pandas()  # Enable tqdm progress_apply
df['clean_text'] = df['text'].progress_apply(preprocess_text)  # Use progress_apply instead of apply

# Encode labels as integers
label_to_id = {label: idx for idx, label in enumerate(set(df['category']))}
id_to_label = {idx: label for label, idx in label_to_id.items()}
df['label_encoded'] = df['category'].apply(lambda x: label_to_id[x])

# Define a custom dataset for PyTorch
class NewsDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        embedding = self.embeddings[idx]
        label = self.labels[idx]
        return torch.tensor(embedding, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Custom collate function to handle padding
def collate_fn(batch):
    embeddings, labels = zip(*batch)
    embeddings = torch.nn.utils.rnn.pad_sequence(
        [emb[:512] for emb in embeddings],  # Ensure no sequence exceeds max_length
        batch_first=True,
        padding_value=0.0
    )
    labels = torch.tensor(labels, dtype=torch.long)
    return embeddings, labels

# Define the BiLSTM model
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, dropout=0.5):
        super(BiLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiply by 2 for bidirectional

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        avg_pool = torch.mean(lstm_out, dim=1)  # Global average pooling
        out = self.dropout(avg_pool)
        out = self.fc(out)
        return out

# Hyperparameters
input_dim = 768  # BERT embedding size
hidden_dim = 64  # Changed to 64 units as per the description
output_dim = len(label_to_id)
num_layers = 1  # Single-layer LSTM
dropout = 0.5
learning_rate = 0.001
num_epochs = 100  # Increased to 100 epochs
batch_size = 16

# Load pre-trained BERT tokenizer and model
print("Loading pre-trained BERT model...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(device)  # Move BERT model to GPU

# Function to compute token-level embeddings using BERT
def get_token_embeddings(text, tokenizer, model, max_length=512):
    # Tokenize the text
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        padding='max_length',  # Pad to max_length
        max_length=max_length
    ).to(device)  # Move inputs to GPU
    
    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Return token embeddings (last_hidden_state)
    return outputs.last_hidden_state.squeeze(0).cpu().numpy()  # Move embeddings back to CPU for storage

# Convert documents to token embeddings
print("Converting data to BERT token embeddings...")
X_embeddings = [get_token_embeddings(text, tokenizer, bert_model) for text in tqdm(df['clean_text'])]
y_labels = df['label_encoded'].values

# 5-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X_embeddings), 1):
    print(f"Training on Fold {fold}...")
    
    # Split data into training and testing sets for this fold
    X_train_embeddings = [X_embeddings[i] for i in train_idx]
    X_test_embeddings = [X_embeddings[i] for i in test_idx]
    y_train_encoded = y_labels[train_idx]
    y_test_encoded = y_labels[test_idx]

    # Create datasets and dataloaders
    train_dataset = NewsDataset(X_train_embeddings, y_train_encoded)
    test_dataset = NewsDataset(X_test_embeddings, y_test_encoded)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # Initialize the model, loss function, and optimizer
    model = BiLSTMClassifier(input_dim, hidden_dim, output_dim, num_layers, dropout).to(device)  # Move model to GPU
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for embeddings, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
            embeddings, labels = embeddings.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Fold {fold}, Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

    # Evaluate the model
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for embeddings, labels in tqdm(test_loader, desc="Testing"):
            embeddings, labels = embeddings.to(device), labels.to(device)  # Move data to GPU
            outputs = model(embeddings)
            _, preds = torch.max(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())  # Move predictions back to CPU for metrics
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"\nFold {fold}, Accuracy: {accuracy}")
    fold_accuracies.append(accuracy)

# Final Results
print("\nFinal Results:")
print(f"Average Accuracy across 5 folds: {np.mean(fold_accuracies):.4f}")

Using device: cuda
Preprocessing text...


100%|██████████| 6872/6872 [12:42<00:00,  9.02it/s] 


Loading pre-trained BERT model...
Converting data to BERT token embeddings...


100%|██████████| 6872/6872 [02:47<00:00, 40.93it/s]


Training on Fold 1...


...

Fold 5, Epoch 51, Loss: 0.04522318870616335


Epoch 52/100: 100%|██████████| 344/344 [00:04<00:00, 70.84it/s]


Fold 5, Epoch 52, Loss: 0.012835351186367916


Epoch 53/100:  10%|▉         | 34/344 [00:00<00:04, 68.52it/s]


KeyboardInterrupt: 